# Instructions
The following code was designed in order to load in and score motion/freezing data from video files. It allows for cropping of the video frame to reduce influence of miniscope/optogenetic cables.  After initally loading in the video, the user is able to crop the video frame by adjusting a single variable.  Motion is then detected by capturing the number of pixels whose frame by frame grayscale change value exceeds a threshold (determined using FreezeAnalysis_Calibration.ipynb).  Freezing is then assessed by calculating when motion drops below a user-defined threshold.  The user is able to visualize raw video, animal motion, and whether an animal is deemed to be freezing in order optimize parameters.  Final output is a csv which provides frame by frame motion and freezing across the session.  Additionally, summary file can be generated which allows user to get motion/freezing for specified time bins.  With the exception of setting the file path and small number of parameters (ie frames per second in video, freezing threshold), user can run through code.  Once parameters are found that work well, batch processing of multiple videos can be performed using FreezeAnalysis_BatchProcess.ipynb.

### Package Requirements
Please see instructions under repository README for package requirements and install instructions.

---
# 1. Load Necessary Packages

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import BehaviorTracking_Functions as fz
plt.gray()

---
# 2. User Sets Directory and File Information

In [ ]:
dpath = "/Users/ZP/Desktop/Videos" # directory containing file
file = "Test.mpg" #filename.  will take mpg and wmv files but maybe more.  only mpg1 has been extensively tested.
fps = 30 #frames per second


---
# 3. Load Video Information.  Display First Frame

In [ ]:
#Get full path and first frame
fpath,gray = fz.Load_First(dpath,file)

#Present frame
plt.figure(figsize=(8,8))
plt.title('first frame')
plt.imshow(gray)

---
# 4. User Selects Cropping Parameters
Here, upper portion of video to be cropped can be set.  Use y-axis value from previous cell. Be sure to set ycrop=0 if no cropping desired


In [ ]:
#set ycrop
ycrop = 130

#Displays result
plt.gray()
plt.figure(figsize=(8,8))
plt.title('Cropped Frame. ycrop=' + str(ycrop))
plt.imshow(gray[ycrop:,:])

---
# 5. Analyze Motion Across Session

### 5a. User Sets Parameters for Motion Analysis

In [ ]:
mt_cutoff = 10 #grayscale difference value required for pixel to be counted as changing
SIGMA = 1 #this is sigma used for gaussian smoothing of image.  Used to reduce influence of frame by frame pixel jitter. 1 works well but could increased slightly if noise is detected. 


### 5b. Detect Motion
Here, code loops through all frames and detects number of pixels whose grayscale change exceeds cutoff per frame.  Motion vector is populated with these values.

In [ ]:
Motion = fz.Measure_Motion(fpath,ycrop,mt_cutoff,SIGMA)  #calls function to get motion

### 5c. Plot Motion Across Session
User can alter y coordinates as desired

In [ ]:
#set y coordinates
ymin = 0
ymax = 500

#plot figure
plt.figure(figsize=(15,4))
plt.ylim(ymin,ymax)
plt.title('Motion Across Session')
plt.xlabel('Frame')
plt.ylabel('Pixel Change')
plt.plot(Motion)

---
# 6. Analyze Session Freezing

### 6a. User Selects Freezing Parameters

In [ ]:
FreezeThresh = 200 #Upper bound for freezing, in frame-by-frame pixels changed
MinDuration = 1 #Number of seconds motion must be below threshold to begin accruing freezing
MinDuration = MinDuration * fps

### 6b. Measure Freezing

In [ ]:
Freezing = fz.Measure_Freezing(Motion,FreezeThresh,MinDuration)  

### 6c. Plot Freezing

In [ ]:
print('Average Freezing: '+ str(np.average(Freezing)) + ' %')

#Plot Freezing
plt.figure(figsize=(15,4))
plt.ylim(0,101)
plt.title('Freezing Across Session')
plt.xlabel('Frame')
plt.ylabel('Freezing')
plt.plot(Freezing)

---
# 7. (Optional) Display Section of Video with Motion and Freezing
After analyzing freezing a section of the video can be replayed and the animal's state - "Active/Freezing" - will be displayed.  

### 7a. User Selects Portion of Video to be Played

In [ ]:
#Video parameters
start = 2100 #start point of video segment in frames.  0 if beginning of video.
end = 2400 #end point of video segment in frames.  qqqqqqthis is NOT the duration of the segment
img_scale = 2 # #Scale image by this number.5 will reduce original video size 1/2, 2 will double it, etc.
save_video=False #Option to save video if desired.  Currently will be saved at 20 fps even if video is something else

### 7b. Play video.  Select video frame and hold 'q' to quit video

In [ ]:
fz.PlayVideo(fpath,fps,start,end,img_scale,save_video,Freezing,mt_cutoff,ycrop,SIGMA)

---
# 8. (Optional) Save Frame-By-Frame Motion

In [ ]:
fz.SaveData(file,fpath,Motion,Freezing,fps,mt_cutoff,FreezeThresh,MinDuration)

---
# 9. (Optional) Create Binned Summary Report and Save
The code below allows user to either save a csv containing summary data for user-defined bins (e.g. freezing in each minute) or a session-wide average.

### 9a. User Defines Bins

In [ ]:
Use_Bins = True #Define whether user specified summary bins are to bew used.  True/False.  Set to False if you just want average for full video.

#USER SETS BIN INFORMATION
Bin_Names = ['avg','1','2','3','4','5'] #Create list of bin names. Must be in single/double quotes (eg: ['avg','1','2'])
Bin_Start = [0,0,60,120,180,240] #provide list of bin start times, in seconds (eg: [0,0,60])
Bin_Stop = [300,60,120,180,240,300] #provide list of bin end times, in seconds (eg: [120,60,120])

#Check requirement that lists be equal length
fz.Check(Bin_Names,Bin_Start,Bin_Stop) 

### 9b. Create Summary and Save

In [ ]:
summary = fz.Summarize(file,Motion,Freezing,Bin_Names,Bin_Start,Bin_Stop,fps,mt_cutoff,FreezeThresh,MinDuration,Use_Bins)
sumpath_out = dpath + "/" + 'SummaryStats.csv'
summary.to_csv(sumpath_out)
summary